<a href="https://colab.research.google.com/github/vedika1509/Data-Structure-Using-C/blob/main/new_uc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jiamingzhang94/Unlearnable-Clusters.git

Cloning into 'Unlearnable-Clusters'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 101 (delta 45), reused 74 (delta 25), pack-reused 0
Receiving objects: 100% (101/101), 1.33 MiB | 20.95 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install -q tfds-nightly tensorflow matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import requests
import zipfile
import os


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 19.6 MB/s eta 0:00:00


In [12]:
import shutil

In [13]:
!pip install ruamel.yaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 19.9 MB/s eta 0:00:00


In [14]:
from ruamel.yaml import YAML

In [15]:
!pip install ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 1.8 MB/s eta 0:00:00


In [16]:
!pip install logger

  Preparing metadata (setup.py) ... done
  Created wheel for logger: filename=logger-1.4-py3-none-any.whl size=1758 sha256=f9119464f4d4f9c1830e9e1f75b9bcd484c0ab7e74c374111f36b121b17adf07
  Stored in directory: /root/.cache/pip/wheels/fb/19/7b/09fc73f7503166eaf7f31b4aa0095b7f78af2ec0898e1f8312
Successfully built logger


In [17]:
import torchvision
import torch
from torchvision import transforms

In [18]:
from torch.utils.data import Dataset

from PIL import Image
import torch

In [19]:
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=490c960bfdcb4b4b80e478184e33bc72b836b4c4a5fc7b109a632390814bfd88
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [20]:
yaml = YAML(typ='rt')

In [21]:
# Source and destination file paths
source_file = '/content/Unlearnable-Clusters/logger/metricLogger.py'
destination_directory = '/usr/local/lib/python3.10/dist-packages/logger'

# Ensure destination directory exists
os.makedirs(destination_directory, exist_ok=True)

# Copy the file from source to destination
shutil.copy(source_file, destination_directory)

print("File copied successfully!")

File copied successfully!


In [22]:
from torch.utils.data import DataLoader
import torch.nn.functional as F
from pathlib import Path

from logger import metricLogger
# from utils import get_surrogate, normalize_list

# from dataset.dataFolder import DataFolderWithLabel, DataFolderWithOneClass

from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans
import json


In [23]:
import os
import urllib.request
import tarfile

# Define the URL to download the dataset
url = 'http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz'

# Define the directory to save the downloaded file
download_dir = '/content/data'

# Create the directory if it doesn't exist
os.makedirs(download_dir, exist_ok=True)

# Download the dataset
print("Downloading Oxford-IIIT Pet dataset...")
urllib.request.urlretrieve(url, os.path.join(download_dir, 'images.tar.gz'))
print("Download complete!")

# Extract the downloaded file
print("Extracting dataset...")
with tarfile.open(os.path.join(download_dir, 'images.tar.gz'), 'r:gz') as tar:
    tar.extractall(download_dir)
print("Extraction complete!")

# Remove the downloaded file
os.remove(os.path.join(download_dir, 'images.tar.gz'))

print("Dataset downloaded and extracted successfully!")


Download complete!
Extracting dataset...
Extraction complete!
Dataset downloaded and extracted successfully!


In [24]:
# from google.colab import drive
# import shutil
# import os

# # Step 1: Zip the directory
# directory_to_download = '/content/data'
# shutil.make_archive(directory_to_download, 'zip', directory_to_download)

# # Step 2: Mount Google Drive
# drive.mount('/content/drive')

# # Step 3: Move the zipped directory to Google Drive
# shutil.move(f"{directory_to_download}.zip", "/content/drive/My Drive/")

# # Step 4: Download from Google Drive
# # After running the above code, go to your Google Drive, find the zipped directory, right-click, and select "Download".


In [25]:
import os
import shutil
import random

# Define the source directory containing the downloaded images
source_dir = '/content/data/images'

# Define the destination directories for train and test data
train_dir = '/content/data/train'
test_dir = '/content/data/test'

# Create the train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Function to move a file from source to destination
def move_file(source, destination):
    shutil.move(source, destination)

# Iterate over each image file in the source directory
image_files = os.listdir(source_dir)
for image_file in image_files:
    # Get the class label from the image file name
    label = image_file.split('_')[0]

    # Create corresponding train and test directories for the label
    train_label_dir = os.path.join(train_dir, label)
    test_label_dir = os.path.join(test_dir, label)
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(test_label_dir, exist_ok=True)

    # Determine whether the image should be used for training or testing
    if random.random() < 0.8:  # 80% chance for training
        destination_dir = train_label_dir
    else:  # 20% chance for testing
        destination_dir = test_label_dir

    # Move the image file to the appropriate directory
    move_file(os.path.join(source_dir, image_file), os.path.join(destination_dir, image_file))

print("Dataset divided into train and test directories successfully!")


Dataset divided into train and test directories successfully!


In [26]:

# # Define the folder containing the images
# folder_path = '/content/data/images'

# # Get the list of all files in the folder
# files = os.listdir(folder_path)

# # Filter out only JPEG images and sort them
# jpeg_images = sorted([file for file in files if file.endswith('.jpg')])

# # Get the first 5000 images
# images_to_keep = jpeg_images[:5000]

# # Delete the rest of the images and non-JPEG files
# for file in files:
#     if file not in images_to_keep or not file.endswith('.jpg'):
#         os.remove(os.path.join(folder_path, file))

# print("Operation completed successfully. Only the first 5000 JPEG images are kept in the folder.")


In [27]:

# # Source directory containing all the images
# source_dir = "/content/data/images"

# # Destination directories for train and test
# train_dir = "/content/data/train"
# test_dir = "/content/data/test"

# # Create train and test directories if they don't exist
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)

# # Get a list of all image files in the source directory
# image_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

# # Sort the image files to ensure consistent splitting
# image_files.sort()

# # Split the image files into train and test sets
# train_files = image_files[:4000]
# test_files = image_files[4000:]

# # Move train files to the train directory
# for file in train_files:
#     src = os.path.join(source_dir, file)
#     dst = os.path.join(train_dir, file)
#     shutil.move(src, dst)

# # Move test files to the test directory
# for file in test_files:
#     src = os.path.join(source_dir, file)
#     dst = os.path.join(test_dir, file)
#     shutil.move(src, dst)

# print("Data split and saved successfully!")


In [28]:
# import shutil

# # Specify the directory path to be deleted
# directory_to_delete = '/content/data'

# # Attempt to remove the directory and its contents
# try:
#     shutil.rmtree(directory_to_delete)
#     print(f"Directory '{directory_to_delete}' and its contents have been successfully deleted.")
# except OSError as e:
#     print(f"Error: {directory_to_delete} : {e.strerror}")


In [75]:
# import yaml

# # Load the YAML file
# with open('/content/Unlearnable-Clusters/config/data.yaml', 'r') as file:
#     data = yaml.safe_load(file)

# # Modify the values of 'train' and 'test'
# data['pets']['train'] = '/content/data/train'
# data['pets']['test'] = '/content/data/test'

# data['cars']['train'] = '/content/drive/MyDrive/car196/train'
# data['cars']['test'] = '/content/drive/MyDrive/car196/test'

# # Save the updated YAML file
# with open('updated_data.yaml', 'w') as file:
#     yaml.dump(data, file)

# print("YAML file updated successfully.")


In [76]:

# Load YAML file
yaml = YAML()
with open("/content/Unlearnable-Clusters/config/preprocess.yaml", 'r') as file:
    config = yaml.load(file)

# Access the configuration data
print(config)

{'rename': {'data_config': '/content/Unlearnable-Clusters/config/data.yaml', 'dataset': 'pets'}, 'rename_pets': {'data_config': '/content/Unlearnable-Clusters/config/data.yaml', 'dataset': 'pets'}, 'cluster_pets_rn50_cluster10': {'data_config': '/content/Unlearnable-Clusters/config/data.yaml', 'dataset': 'pets', 'model': 'RN50', 'checkpoint': '/content/Unlearnable-Clusters/checkpoints/RN50_imagenet.pth', 'normalize': 'imagenet', 'num_classes': 1000, 'num_clusters': 10, 'output_dir': '/content/Unlearnable-Clusters/output/cluster'}, 'cluster_pets_cliprn50_cluster10': {'data_config': '/content/Unlearnable-Clusters/config/data.yaml', 'dataset': 'pets', 'model': 'CLIPRN50', 'checkpoint': 'checkpoints/RN50_clip.pth', 'normalize': 'clip', 'num_classes': 1000, 'num_clusters': 10, 'output_dir': '/content/Unlearnable-Clusters/output/cluster'}}


In [77]:

# Load YAML file
yaml = YAML()
yaml_file_path = "/content/Unlearnable-Clusters/config/preprocess.yaml"  # Replace with the actual path to your YAML file
with open(yaml_file_path, 'r') as file:
    config = yaml.load(file)

# Check if the function 'rename' exists in the YAML file
if 'rename' in config:
    # Execute the 'rename' function
    rename_config = config['rename']
    # Implement the logic to execute the 'rename' function here

    print("Function 'rename' executed successfully.")
else:
    print("Function 'rename' not found in the YAML file.")


Function 'rename' executed successfully.


In [78]:
!pip install PyYAML

In [79]:
import yaml
with open("/content/Unlearnable-Clusters/config/preprocess.yaml", 'r') as file:
    config = yaml.load(file, Loader=yaml.Loader)

In [80]:
from ruamel.yaml import YAML

# Assuming args.config


In [81]:
    #     # Load YAML file using yaml.safe_load()
    # with open(args.config, 'r') as file:
    #     config = yaml.safe_load(file)[args.function]

    # # Load data config
    # with open(config['data_config'], 'r') as file:
    #     data_config = yaml.safe_load(file)[config['dataset']]

#     from ruamel.yaml import YAML

# # Create an instance of the YAML class
# yaml = YAML(typ='rt')

# # Load YAML data from file
# with open(config['data_config'], 'r') as file:
#     data_config = yaml.load(file)[config['dataset']]


In [82]:
!pip install PyYAML
import yaml
stream = open('/content/Unlearnable-Clusters/config/preprocess.yaml', 'r')
data = yaml.full_load(stream)
stream.close()

In [83]:
!pip install dataset

In [84]:
!python /content/Unlearnable-Clusters/dataset/dataCluster.py

In [85]:
!python /content/Unlearnable-Clusters/dataset/dataFolder.py

In [86]:
!python /content/Unlearnable-Clusters/checkpoints/__init__.py

In [87]:
from ruamel.yaml import YAML

In [88]:
from torch.utils.data import Dataset
import os
from PIL import Image
import torch
import torchvision.transforms as transforms


def is_image_file(filename):
    IMG_EXTENSIONS = [
        '.jpg', '.JPG', '.jpeg', '.JPEG',
        '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP', '.tiff'
    ]
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)


class DataFolderWithLabel(Dataset):
    def __init__(self, root, pred_idx, transform=None):
        self.labels = []
        self.images = []
        self.transform = transform

        for class_name in sorted(os.listdir(root)):
            label = int(class_name)
            for file_name in sorted(os.listdir(os.path.join(root, class_name))):
                if not is_image_file(file_name):
                    continue
                self.images.append(os.path.join(root, class_name, file_name))
                self.labels.append(label)

        if pred_idx is None:
            self.pred_idx = self.labels
        else:
            self.pred_idx = pred_idx

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        return image, label, self.pred_idx[idx]


class DataFolderWithClassNoise(Dataset):
    def __init__(self, root, pred_idx, noise=None, resize_type='img'):
        self.labels = []
        self.images = []
        self.resize_type = resize_type
        if self.resize_type == 'img':
            self.transform = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
            ])
        else:
            self.transform = transforms.Compose([
                transforms.ToTensor(),
            ])

        for class_name in sorted(os.listdir(root)):
            label = int(class_name)
            for file_name in sorted(os.listdir(os.path.join(root, class_name))):
                if not is_image_file(file_name):
                    continue
                self.images.append(os.path.join(root, class_name, file_name))
                self.labels.append(label)

        if noise is None:
            self.noise = torch.zeros((1, 3, 112, 112))
            self.noise.uniform_(0, 1)
            self.noise = self.noise.repeat(self.num_classes, 1, 1, 1)
        else:
            self.noise = noise

        self.pred_idx = pred_idx

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        if self.resize_type == 'img':
            image = torch.clamp(image + self.noise[self.pred_idx[idx]], 0, 1)
        else:
            img_size = image.size()
            noise = torch.nn.functional.interpolate(self.noise[self.pred_idx[idx]].unsqueeze(0), (img_size[1], img_size[2]))
            image = torch.clamp(image + noise, 0, 1)
        return image, label, self.pred_idx[idx]


class DataFolderWithOneClass(Dataset):
    def __init__(self, root, cluster_idx, pred_idx, transform=None, offset=0):
        self.images = []
        self.label = []
        self.transform = transform

        for class_name in sorted(os.listdir(root)):
            label = int(class_name) - offset
            for file_name in sorted(os.listdir(os.path.join(root, class_name))):
                if not is_image_file(file_name):
                    continue
                self.images.append(os.path.join(root, class_name, file_name))
                self.label.append(label)

        self.pred_idx = pred_idx
        self.cluster_idx = cluster_idx

        self.images = [self.images[i] for i, eq in enumerate(pred_idx == cluster_idx) if eq]
        self.label = [self.label[i] for i, eq in enumerate(pred_idx == cluster_idx) if eq]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')

        if self.transform:
            image = self.transform(image)
        return image, self.label[idx], self.cluster_idx


In [105]:
!python /content/Unlearnable-Clusters/preprocess.py --config /content/Unlearnable-Clusters/config/preprocess.yaml -f cluster_pets_rn50_cluster10
# the cluster file will be stored in {output_dir}/{dataset}_{surrogate_model}_cluster{num_clusters}.pth
# e.g., output/cluster/pets_rn50_cluster10.pth

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Traceback (most recent call last):
  File "/content/Unlearnable-Clusters/preprocess.py", line 157, in <module>
    data_cluster(args, config)
  File "/content/Unlearnable-Clusters/preprocess.py", line 71, in data_cluster
    sd = torch.load(config['checkpoint'], map_location='cpu')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 986, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 435, in _open_file_like

In [ ]:
from ruamel.yaml import YAML
import argparse
import torch
import numpy as np
import os
import shutil
from pathlib import Path
from logger import metricLogger
# from utils import get_surrogate, normalize_list
# from dataset.dataFolder import DataFolderWithLabel, DataFolderWithOneClass
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans
import json
from torchvision import transforms
from torch.utils.data import DataLoader
# !pip install PyYAML
import yaml

def dataset_median_embedding(args, config):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

    normalize = normalize_list[args.normalize]

    num_classes = len(os.listdir(config[args.dataset]['test']['path']))
    net = get_model(args.model, num_classes).eval().to(args.device)
    sd = torch.load(args.checkpoint, map_location='cpu')
    sd = sd.get('state_dict', sd)
    net.load_state_dict(sd)

    result = []

    for i in range(num_classes):
        train_dataset = DataFolderWithOneClass(config[args.dataset]['train']['path'], i, transform)
        train_loader = DataLoader(train_dataset, batch_size=64, num_workers=8)

        features = []
        def hook(layer, inp, out):
            features.append(inp[0].cpu())
        net.fc.register_forward_hook(hook)

        for images, labels in train_loader:
            images, labels = images.to(args.device), labels.to(args.device)
            with torch.no_grad():
                net(normalize(images))

        features = torch.cat(features, dim=0)
        dis = squareform(pdist(features.numpy())).sum(axis=1)
        dis = torch.tensor(dis)
        idx = torch.argmin(dis)
        result.append(features[idx])

    torch.save(result, os.path.join(args.output_dir, f'{args.dataset}_{args.model}_median_embed.pth'))


def data_cluster(args, config):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

    normalize = normalize_list[config['normalize']]

    train_dataset = DataFolderWithLabel(config['dataset']['config']['train'], transform)
    train_loader = DataLoader(train_dataset, batch_size=64, num_workers=4)

    sd = torch.load(config['checkpoint'], map_location='cpu')
    sd = sd.get('state_dict', sd)
    net = get_model(config['model'], config['num_classes']).eval().to(args.device)
    net.load_state_dict(sd)

    features = []

    def hook(layer, inp, out):
        features.append(inp[0].cpu())

    net.fc.register_forward_hook(hook)

    for images, labels in train_loader:
        images, labels = images.to(args.device), labels.to(args.device)
        with torch.no_grad():
            net(normalize(images))

    features = torch.cat(features, dim=0)
    classifier = KMeans(n_clusters=config['num_clusters'])
    pred_idx = classifier.fit_predict(features.numpy())

    test_dataset = DataFolderWithLabel(config['dataset']['config']['test'], transform)
    test_loader = DataLoader(test_dataset, batch_size=256, num_workers=8)

    features = []
    for images, labels in test_loader:
        images, labels = images.to(args.device), labels.to(args.device)
        with torch.no_grad():
            net(normalize(images))

    features = torch.cat(features, dim=0)
    pred_idx_test = classifier.predict(features.numpy())

    result = {'pred_idx': torch.tensor(pred_idx), 'pred_idx_test': pred_idx_test, 'centers': torch.tensor(classifier.cluster_centers_)}
    num_clusters = result['centers'].shape[0]
    torch.save(result, os.path.join(config['output_dir'], f"{config['dataset']['name']}_{config['model'].lower()}_cluster{num_clusters}.pth"))


def datafolder_rename(args, config):
    train_dir = config['dataset']['config']['train']
    test_dir = config['dataset']['config']['test']

    class_list = sorted(os.listdir(train_dir))
    new_class = list(map(str, range(len(class_list))))

    name_mapping = {}
    name_mapping['map'] = {k: v for k, v in zip(class_list, new_class)}
    name_mapping['inv'] = {k: v for k, v in zip(new_class, class_list)}

    json.dump(name_mapping, open(os.path.join(train_dir, '..', 'name_mapping.json'), 'w+'))

    os.rename(train_dir, train_dir+'_raw')
    os.rename(test_dir, test_dir+'_raw')
    os.mkdir(train_dir)
    os.mkdir(test_dir)
    for i in range(len(class_list)):
        shutil.copytree(os.path.join(train_dir+'_raw', class_list[i]), os.path.join(train_dir, new_class[i]))
        shutil.copytree(os.path.join(test_dir+'_raw', class_list[i]), os.path.join(test_dir, new_class[i]))

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--device', type=int, default=0)
    parser.add_argument('--config', type=str)

    parser.add_argument('--seed', type=int, default=42)

    parser.add_argument('--function', '-f', type=str)

    args = parser.parse_args()

    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    np.random.seed(args.seed)

    # Load YAML file using yaml.load()
    with open(args.config, 'r') as file:
        config = yaml.full_load(file)

    # Access the required data
    config = config[args.function]

    if args.function.startswith('median'):
        Path(config['output_dir']).mkdir(parents=True, exist_ok=True)
        dataset_median_embedding(args, config)
    elif args.function.startswith('cluster'):
        Path(config['output_dir']).mkdir(parents=True, exist_ok=True)
        data_cluster(args, config)
    elif args.function.startswith('rename'):
        datafolder_rename(args, config)


In [ ]:
class DataFolderWithLabel(Dataset):
    def __init__(self, root, transform=None):
        self.labels = []
        self.images = []
        self.transform = transform

        # Dictionary to map directory names to integer labels
        class_mapping = {
    'Abyssinian': 0,
    'Bengal': 1,
    'Birman': 2,
    'Bombay': 3,
    'British_Shorthair': 4,
    'Egyptian_Mau': 5,
    'Maine_Coon': 6,
    'Persian': 7,
    'Ragdoll': 8,
    'Russian_Blue': 9,
    'Siamese': 10,
    'Sphynx': 11
}


        for class_name in sorted(os.listdir(root)):
            label = class_mapping.get(class_name)
            if label is not None:
                for file_name in sorted(os.listdir(os.path.join(root, class_name))):
                    if not is_image_file(file_name):
                        continue
                    self.images.append(os.path.join(root, class_name, file_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
DataFolderWithLabel('/content/data/train')

In [ ]:
from ruamel.yaml import YAML  # Import the YAML module

# # Create a YAML instance
# yaml = YAML(typ='rt')

# # Load the YAML file
with open('/content/Unlearnable-Clusters/config/preprocess.yaml', 'r') as file:
    config = yaml.load(file)

# # Access the required data
config = config['cluster_pets_rn50_cluster10']

# Execute preprocessing script with the specified YAML configuration
!python /content/Unlearnable-Clusters/preprocess.py --config /content/Unlearnable-Clusters/config/preprocess.yaml -f cluster_pets_rn50_cluster10


Traceback (most recent call last):
  File "/content/Unlearnable-Clusters/preprocess.py", line 9, in <module>
    from utils import get_surrogate, normalize_list
  File "/content/Unlearnable-Clusters/utils/__init__.py", line 1, in <module>
    from .util import adjust_learning_rate, get_surrogate, get_target, normalize_list
  File "/content/Unlearnable-Clusters/utils/util.py", line 1, in <module>
    from models import *
ModuleNotFoundError: No module named 'models'


In [ ]:
# from ruamel.yaml import YAML
# import argparse
# import torch
# import numpy as np
# import os
# import shutil
# from pathlib import Path
# from logger import MetricLogger
# from utils import get_surrogate, normalize_list
# from dataset.dataFolder import DataFolderWithLabel, DataFolderWithOneClass
# from scipy.spatial.distance import pdist, squareform
# from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans
# import json
# from torchvision import transforms
# from torch.utils.data import DataLoader
# # !pip install PyYAML
# import yaml

# def dataset_median_embedding(args, config):
#     transform = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#     ])

#     normalize = normalize_list[args.normalize]

#     num_classes = len(os.listdir(config[args.dataset]['test']['path']))
#     net = get_model(args.model, num_classes).eval().to(args.device)
#     sd = torch.load(args.checkpoint, map_location='cpu')
#     sd = sd.get('state_dict', sd)
#     net.load_state_dict(sd)

#     result = []

#     for i in range(num_classes):
#         train_dataset = DataFolderWithOneClass(config[args.dataset]['train']['path'], i, transform)
#         train_loader = DataLoader(train_dataset, batch_size=64, num_workers=8)

#         features = []
#         def hook(layer, inp, out):
#             features.append(inp[0].cpu())
#         net.fc.register_forward_hook(hook)

#         for images, labels in train_loader:
#             images, labels = images.to(args.device), labels.to(args.device)
#             with torch.no_grad():
#                 net(normalize(images))

#         features = torch.cat(features, dim=0)
#         dis = squareform(pdist(features.numpy())).sum(axis=1)
#         dis = torch.tensor(dis)
#         idx = torch.argmin(dis)
#         result.append(features[idx])

#     torch.save(result, os.path.join(args.output_dir, f'{args.dataset}_{args.model}_median_embed.pth'))


# def data_cluster(args, config):
#     transform = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#     ])

#     normalize = normalize_list[config['normalize']]

#     train_dataset = DataFolderWithLabel(config['dataset']['config']['train'], transform)
#     train_loader = DataLoader(train_dataset, batch_size=64, num_workers=4)

#     sd = torch.load(config['checkpoint'], map_location='cpu')
#     sd = sd.get('state_dict', sd)
#     net = get_model(config['model'], config['num_classes']).eval().to(args.device)
#     net.load_state_dict(sd)

#     features = []

#     def hook(layer, inp, out):
#         features.append(inp[0].cpu())

#     net.fc.register_forward_hook(hook)

#     for images, labels in train_loader:
#         images, labels = images.to(args.device), labels.to(args.device)
#         with torch.no_grad():
#             net(normalize(images))

#     features = torch.cat(features, dim=0)
#     classifier = KMeans(n_clusters=config['num_clusters'])
#     pred_idx = classifier.fit_predict(features.numpy())

#     test_dataset = DataFolderWithLabel(config['dataset']['config']['test'], transform)
#     test_loader = DataLoader(test_dataset, batch_size=256, num_workers=8)

#     features = []
#     for images, labels in test_loader:
#         images, labels = images.to(args.device), labels.to(args.device)
#         with torch.no_grad():
#             net(normalize(images))

#     features = torch.cat(features, dim=0)
#     pred_idx_test = classifier.predict(features.numpy())

#     result = {'pred_idx': torch.tensor(pred_idx), 'pred_idx_test': pred_idx_test, 'centers': torch.tensor(classifier.cluster_centers_)}
#     num_clusters = result['centers'].shape[0]
#     torch.save(result, os.path.join(config['output_dir'], f"{config['dataset']['name']}_{config['model'].lower()}_cluster{num_clusters}.pth"))


# def datafolder_rename(args, config):
#     train_dir = config['dataset']['config']['train']
#     test_dir = config['dataset']['config']['test']

#     class_list = sorted(os.listdir(train_dir))
#     new_class = list(map(str, range(len(class_list))))

#     name_mapping = {}
#     name_mapping['map'] = {k: v for k, v in zip(class_list, new_class)}
#     name_mapping['inv'] = {k: v for k, v in zip(new_class, class_list)}

#     json.dump(name_mapping, open(os.path.join(train_dir, '..', 'name_mapping.json'), 'w+'))

#     os.rename(train_dir, train_dir+'_raw')
#     os.rename(test_dir, test_dir+'_raw')
#     os.mkdir(train_dir)
#     os.mkdir(test_dir)
#     for i in range(len(class_list)):
#         shutil.copytree(os.path.join(train_dir+'_raw', class_list[i]), os.path.join(train_dir, new_class[i]))
#         shutil.copytree(os.path.join(test_dir+'_raw', class_list[i]), os.path.join(test_dir, new_class[i]))

# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--device', type=int, default=0)
#     parser.add_argument('--config', type=str)

#     parser.add_argument('--seed', type=int, default=42)

#     parser.add_argument('--function', '-f', type=str)

#     args = parser.parse_args()

#     torch.manual_seed(args.seed)
#     torch.cuda.manual_seed(args.seed)
#     np.random.seed(args.seed)

#     # Load YAML file using yaml.load()
#     with open(args.config, 'r') as file:
#         config = yaml.full_load(file)

#     # Access the required data
#     config = config[args.function]

#     if args.function.startswith('median'):
#         Path(config['output_dir']).mkdir(parents=True, exist_ok=True)
#         dataset_median_embedding(args, config)
#     elif args.function.startswith('cluster'):
#         Path(config['output_dir']).mkdir(parents=True, exist_ok=True)
#         data_cluster(args, config)
#     elif args.function.startswith('rename'):
#         datafolder_rename(args, config)
